In [1]:
import numpy as np
import math
import os
import gdal
import xarray as xr
from skimage.measure import block_reduce
from ftplib import FTP
import subprocess
import rioxarray
import time as T
import datetime
import ephem

def calc_sunpos(dtime):
    sun = ephem.Sun()
    obs = ephem.Observer()
    obs.date = dtime
    obs.lat = site[0]*math.pi/180.0
    obs.long = site[1]*math.pi/180.0
    sun.compute(obs)
    return np.degrees(sun.az),90.0-np.degrees(sun.alt)
class SGLI_L2_Downloader:
    
    def __init__ (self, User_ID , ProductName, Ver, Date , Tile , Obrit ,ParaVer, Target_path):
        self.User_ID = User_ID
        self.ProductName = ProductName
        self.Ver = Ver
        self.Date = Date
        self.Tile = Tile
        self.Obrit = Obrit
        self.ParaVer = ParaVer
        self.Target_path = Target_path
        
        
    def path (self):
        return '/standard/GCOM-C/GCOM-C.SGLI/L2.LAND.{}/{}/{}/{}/{}'.format(self.ProductName, self.Ver, self.Date[0:4], self.Date[4:6], self.Date[6:8])

    def filename (self):
        return 'GC1SG1_{}{}01D_T{}_L2SG_{}Q_{}00{}.h5'.format(self.Date, self.Obrit, self.Tile, self.ProductName, self.Ver, self.ParaVer)
    
    def login (self):
        ftp_addr = 'ftp.gportal.jaxa.jp'
        f = FTP(ftp_addr)
        f.login(self.User_ID,'anonymous')
        
        return f
    
    def download (self, f):
        remote_filepath = self.path()
        f.cwd(remote_filepath)
        list=f.nlst()
        
        bufsize=1024
        if self.filename() in list:
            data = open ('{}{}'.format(self.Target_path, self.filename()), 'wb')
            filename = 'RETR ' + self.filename()
            f.retrbinary(filename,data.write,bufsize)
            return 'y'
        else:
            return 'n'
    
    def logout (self, f):
        f.quit()
    
def reporjection_GEO(filename,Band,Target_path):
    
    #print(path)
    subprocess.check_output('/media/liwei/Data/GCOM-C_RSRF/SGLI_geo_map_linux.exe {} -d Geometry_data/{} -o {} -s 7.5'.format(filename,Band,Target_path[:-1]), shell=True) 
    
def mkdir(path):
    folder = os.path.exists(path) 
    if not folder:
        os.makedirs(path)
        
    
def SGLI_2_AHI_GEO(filepath):
    geotiff_da = rioxarray.open_rasterio(filepath)
    data = geotiff_da.interp(x=lon_y,y=lat_x,method="nearest")
    data = np.where(data != -32768, data ,np.nan)
    data = data * 0.01
    data = data.reshape(24,24)
#     data = block_reduce(data, block_size=(2,2), func=np.nanmean)
    return data

def SGLI_2_AHI_TIME(filepath):
    geotiff_da = rioxarray.open_rasterio(filepath)
    data = geotiff_da.interp(x=lon_y,y=lat_x,method="nearest")
    data = np.where(data != -32768, data ,np.nan)
    data = data * 0.001
    data = data.reshape(24,24)
#     data = block_reduce(data, block_size=(2,2), func=np.nanmean)
    return data


def month_size(month,YYYY):
    big_month=['01','03','05','07','08','10','12']
    small_month=['04','06','09','11']
    if month in big_month:
        s = 31
    elif month in small_month:
        s = 30
    elif month == '02':
        if (int(YYYY) % 4 == 0 and int(YYYY) % 100 != 0) or int(YYYY) % 400 == 0:
            s = 29
        else:
            s = 28
    return s

def get_site_range(lat,lon):
    crow = int(60 - lat) / 0.04
    ccol = int(lon - 85) / 0.04
    return int(crow-2),int(crow+2),int(ccol-2),int(ccol+2)

def download_tile(date,save_tile_path,tile):
    g = SGLI_L2_Downloader(
        'galiwei ' ,
        'RSRF', 
        '3', 
         date , 
         tile, 
        'D' , 
        '1' ,
         save_tile_path
        )
    f = g.login()
    
    if os.path.exists(save_tile_path + g.filename()):
        return g.filename()
    
    else :
        rc = g.download(f)
        if rc == 'n':
            g.logout(f)
            return 'n'
        else:
            g.logout(f)
            return g.filename()
    
def download_AHI(target_path , date, time, angle):
    
    date1 = date[0:6]
    date2 = date
    AHI_date = date2 + time
    sza_file_name = '{}.sun.zth.fld.4km.bin.bz2'.format(AHI_date)
    saa_file_name = '{}.sun.azm.fld.4km.bin.bz2'.format(AHI_date)
    
    
    if angle == 'sza':
        if os.path.exists(target_path + sza_file_name[:-4]):
            return sza_file_name[:-4]
    
        else:
            ftp = FTP()
            ftp.connect('hmwr829gr.cr.chiba-u.ac.jp', 21)
            ftp.login()
            path = '/gridded/FD/V20190123/{}/4KM/{}/'.format(date1,date2)
            ftp.cwd(path)
            remote_list=ftp.nlst()
            bufsize=1024*1024
            if sza_file_name in remote_list:
                data = open(target_path + sza_file_name,'wb')
                filename='RETR '+ sza_file_name
                ftp.retrbinary(filename,data.write,bufsize)
                ftp.quit()

                return sza_file_name[:-4]
            else:
#                 print('AHI NO DATA')
                return None
        
    elif angle == 'saa':
        if os.path.exists(target_path + saa_file_name[:-4]):
            return saa_file_name[:-4]
    
        else:
            ftp = FTP()
            ftp.connect('hmwr829gr.cr.chiba-u.ac.jp', 21)
            ftp.login()
            path = '/gridded/FD/V20190123/{}/4KM/{}/'.format(date1,date2)
            ftp.cwd(path)
            remote_list=ftp.nlst()
            bufsize=1024*1024
            if saa_file_name in remote_list:
                data = open(target_path + saa_file_name,'wb')
                filename='RETR '+ saa_file_name
                ftp.retrbinary(filename,data.write,bufsize)
                ftp.quit()
                return saa_file_name[:-4]
            else:
#                 print('AHI NO DATA')
                return None
    
def Decompression(t,fn):
    if os.path.exists('{}.bz2'.format(t + fn)):
        p = subprocess.Popen('lbzip2 -d {}.bz2'.format(t + fn),shell=True)
        p.communicate()
        
def ahi_lon_timeoffset(lon): 
    lon_interval = 15 
    UTC_e_lon = lon_interval / 2 
    timeoffset = math.ceil((lon - UTC_e_lon) / lon_interval) 
    return timeoffset 

In [53]:
year = ['2020','2021']


# MM = ['05','06']
DD=['01','02','03','04','05','06','07','08','09','10',\
    '11','12','13','14','15','16','17','18','19','20',\
    '21','22','23','24','25','26','27','28','29','30',\
    '31']
HH=['09','10','11']
MIN=['00','10','20','30','40','50']

band = ['Solar_azimuth_PL', 'Solar_zenith_PL','Sensor_zenith_PL', 'Sensor_azimuth_PL','Obs_time_PL']


# site_name = 'ROI 1 test1'
# site = [-37.9,177.8] # ROI 1
# tile = '1232'

# site_name = 'ROI 1 test2'
# site = [-38.3,177] # ROI 1
# tile = '1231'


# site_name = 'test 2'
# site = [51.2,157,1] # ROI 1
# tile = '0327'

# site_name = 'test 1'
# site = [16.75,96.5] # ROI 1
# tile = '0727'


# site_name = 'ROI 1'
# site = [-38.5,177] # ROI 1
# tile = '1231'

# site_name = 'ROI 2'
# site = [-44.5,170.5] # ROI 2
# tile = '1330'


# site_name = 'ROI 3'
# site = [27,96.5] # ROI 3
# tile = '0626'# ROI 3


# site_name = 'ROI 4'
# site = [32.5,96.5]
# tile = '0526'


# site_name = 'ROI 5'
# site = [39.5,107]
# tile = '0526'

# site_name = 'ROI 6'
# site = [46,114]
# tile = '0425'

# site_name = 'ROI 7'
# site = [35.328,99.052]
# tile = '0526'

# site_name = 'ROI 8'
# site = [42.990,107.043]
# tile = '0425'

# site_name = 'ROI 8'
# site = [41.3,106.5]
# tile = '0425'

site_name = 'ROI 9'
site = [41.7,104.6]
tile = '0425'


# site_name = 'ROI 10'
# site = [49.6,119.4]
# tile = '0425'

# jet = ahi_lon_timeoffset(site[1])

MATCH_RESULT = []
#Save path
reporjection_file = '/media/liwei/Data/GCOM-C_RSRF_Valid/reporjection/'
tile_file = '/media/liwei/Data/GCOM-C_RSRF_Valid/tile/'
AHI_SAA_t = '/media/liwei/Data/AHI_Angle/Solar_azimuth_angle/'
AHI_SZA_t = '/media/liwei/Data/AHI_Angle/Solar_zenith_angle/'



urow,drow,lcol,rcol = get_site_range(site[0],site[1])
lat_x=np.linspace(site[0]+0.06,site[0]-0.055,24)
lon_y=np.linspace(site[1]-0.06,site[1]+0.055,24)


# AHI VZA/VAA Load
with open('/media/liwei/Data/AHI_Angle/AHI_VZA.dat','rb') as fp:
    AHI_VZA = np.frombuffer(fp.read()).reshape(3000,3000)[urow:drow,lcol:rcol]
AHI_VZA = np.nanmean(AHI_VZA)
with open('/media/liwei/Data/AHI_Angle/AHI_VAA.dat','rb') as fp:
    AHI_VAA = np.frombuffer(fp.read()).reshape(3000,3000)[urow:drow,lcol:rcol]
AHI_VAA = np.nanmean(AHI_VAA)

In [55]:

MATCH_RESULT = []
TIME_delta = np.arange(-30,40,10)
TIME_MATCH = datetime.timedelta(minutes = 30)
for YYYY in year:
    if YYYY == '2020':
        MM = ['07','08','09','10','11','12']
    else:
        MM = ['01','02','03','04','05','06']
    for m in range(len(MM)):
        s = month_size(MM[m],YYYY)
        for d in range(s):

            current_result = []
            time_diff_min = datetime.timedelta(hours=5)

            date = YYYY+MM[m]+DD[d]
            print(date)

    #         SGLI file name
            SGLI_VZA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_3001_Sensor_zenith_PL.tif'.format(date,tile)
            SGLI_VAA_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_3001_Sensor_azimuth_PL.tif'.format(date,tile)
            SGLI_TIME_filename = 'GC1SG1_{}D01D_T{}_L2SG_RSRFQ_3001_Obs_time_PL.tif'.format(date,tile)


    #         Reporjection file exist
            if not os.path.exists(reporjection_file + SGLI_TIME_filename):

    #             print('Start SGLI Download and Reporjection.')
                fn = download_tile(date,tile_file,tile)
                if fn == 'n':
                    continue
                for b in band:
                    reporjection_GEO( tile_file+fn , b , reporjection_file)
    #             print('Reporjection Finsihed!')

            if os.path.exists(reporjection_file + SGLI_VZA_filename):       
                SGLI_VZA_PL = SGLI_2_AHI_GEO(reporjection_file + SGLI_VZA_filename)

                SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)

                VZA_Matching_PL = abs(AHI_VZA - SGLI_VZA_PL)
                if VZA_Matching_PL <= 5:
                    SGLI_Obs_time = SGLI_2_AHI_TIME(reporjection_file + SGLI_TIME_filename)
                    SGLI_Obs_time = np.nanmean(SGLI_Obs_time)
                    SGLI_MIN = int(round(math.modf(SGLI_Obs_time)[0],3)*60)
                    SGLI_HH = int(math.modf(SGLI_Obs_time)[1])

                    dtime = datetime.datetime(int(YYYY),int(MM[m]),int(DD[d]),SGLI_HH,SGLI_MIN)
                    SGLI_SAA,SGLI_SZA = calc_sunpos(dtime)
                    AHI_time_old = datetime.datetime.strptime('{}-{}-{} {}:{}'.format(YYYY,MM[m],DD[d],str(SGLI_HH),str(int(round(3/10)))+'0'), '%Y-%m-%d %H:%M')

                    for minu in TIME_delta:
                        delta = datetime.timedelta(minutes = int(minu))
                        d_new = AHI_time_old + delta

                        date_new = d_new.strftime('%Y') + d_new.strftime('%m') + d_new.strftime('%d')
                        TIME = d_new.strftime('%H') + d_new.strftime('%M')

                        AHI_SAA_file_name = download_AHI(AHI_SAA_t , date_new, TIME, 'saa')
                        if AHI_SAA_file_name != None:

                            Decompression(AHI_SAA_t , AHI_SAA_file_name)
        #                     AHI SAA Load
                            with open(AHI_SAA_t + AHI_SAA_file_name, 'rb') as fp:
                                AHI_SAA = np.frombuffer(fp.read(), dtype='>f4').reshape(3000,3000)[urow:drow,lcol:rcol]
                            AHI_SAA = np.mean(AHI_SAA)


                            SGLI_VAA_PL = SGLI_2_AHI_GEO(reporjection_file + SGLI_VAA_filename)
                            SGLI_VAA_PL = SGLI_VAA_PL + 180
                            SGLI_VAA_PL = np.nanmean(SGLI_VAA_PL)

                            SGLI_RAA_PL = abs(SGLI_SAA - SGLI_VAA_PL)

                            if SGLI_RAA_PL > 180:
                                SGLI_RAA_PL = 360 - SGLI_RAA_PL

                            AHI_RAA = abs(AHI_SAA - AHI_VAA)
                            if AHI_RAA > 180:
                                    AHI_RAA = 360 - AHI_RAA 

                            RAA_Matching = abs(SGLI_RAA_PL - AHI_RAA)
    #                         print('AHI_SAA:{} SGLI_SAA:{}'.format(AHI_SAA,SGLI_SAA))
    #                         print('AHI_VAA:{} SGLI_VAA:{}'.format(AHI_VAA,SGLI_VAA_PL))
                            if RAA_Matching < 10:
                                time_diff = abs(dtime - d_new)

                                if time_diff < time_diff_min and time_diff < TIME_MATCH:
                                    time_diff_min = time_diff

                                    AHI_SZA_file_name = download_AHI(AHI_SZA_t , date_new, TIME, 'sza')
                                    if AHI_SZA_file_name != None:

                                        Decompression(AHI_SZA_t,AHI_SZA_file_name)
                                        with open(AHI_SZA_t + AHI_SZA_file_name, 'rb') as fp:
                                            AHI_SZA = np.frombuffer(fp.read(),dtype='>f4').reshape(3000,3000)[urow:drow,lcol:rcol]
                                        AHI_SZA = np.mean(AHI_SZA)

                                    current_result = ['AHI TIME:{}'.format(d_new),
                                                      'SGLI TIME:{}'.format(dtime),
                                                      'TIME DIFF:{}'.format(time_diff_min),
                                                      'VZA Matching:{}'.format(VZA_Matching_PL),
                                                      'SZA Matching:{}'.format(abs(AHI_SZA - SGLI_SZA)),
                                                      'RAA Matching:{}'.format(RAA_Matching),
                                                      'AHI_RAA:{} SGLI_RAA:{}'.format(AHI_RAA,SGLI_RAA_PL),
                                                      'AHI_SZA:{} SGLI_SZA:{}'.format(AHI_SZA,SGLI_SZA),
                                                      'AHI_VZA:{} SGLI_VZA:{}'.format(AHI_VZA,SGLI_VZA_PL),
                                                      ''
                                                      ]
    #         print(current_result)
            if len(current_result) != 0:
                print(current_result)
                MATCH_RESULT.append(current_result)


    with open("{}_{}_PL.txt".format(site_name,YYYY),"w") as f:
        for line in MATCH_RESULT:
            f.writelines('\n'.join(line)) 
            f.writelines('\n')     

20200701
20200702


/tmp/ipykernel_1574303/1319295694.py:39: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)


20200703
20200704


/tmp/ipykernel_1574303/1319295694.py:39: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)
/tmp/ipykernel_1574303/1319295694.py:39: RuntimeWarning: Mean of empty slice
  SGLI_VZA_PL = np.nanmean(SGLI_VZA_PL)


20200705
20200706
20200707


KeyboardInterrupt: 

In [23]:
with open("{}_{}_PL.txt".format(site_name,YYYY),"w") as f:
    for line in MATCH_RESULT:
        f.writelines('\n'.join(line)) 
        f.writelines('\n') 

In [16]:
MATCH_RESULT

[['AHI TIME:2021-03-26 04:10:00',
  'SGLI TIME:2021-03-26 04:37:00',
  'TIME DIFF:0:27:00',
  'VZA Matching:2.758251651483448',
  'SZA Matching:3.6626291208979467',
  'RAA Matching:9.544501947896109',
  'AHI_RAA:22.00741463554631 SGLI_RAA:12.462912687650203',
  'AHI_SZA:37.268516540527344 SGLI_SZA:33.6058874196294',
  'AHI_VZA:60.99991831815011 SGLI_VZA:58.24166666666666',
  ''],
 ['AHI TIME:2021-04-10 04:20:00',
  'SGLI TIME:2021-04-10 04:39:00',
  'TIME DIFF:0:19:00',
  'VZA Matching:1.5612226659761959',
  'SZA Matching:2.7642138788652844',
  'RAA Matching:9.86541017655857',
  'AHI_RAA:21.955321129686936 SGLI_RAA:12.089910953128367',
  'AHI_SZA:30.504444122314453 SGLI_SZA:27.74023024344917',
  'AHI_VZA:60.99991831815011 SGLI_VZA:59.43869565217391',
  ''],
 ['AHI TIME:2021-04-14 04:30:00',
  'SGLI TIME:2021-04-14 04:33:00',
  'TIME DIFF:0:03:00',
  'VZA Matching:3.8997794292612085',
  'SZA Matching:0.8194820662220295',
  'RAA Matching:4.313418189162363',
  'AHI_RAA:25.18594246757756 S